In [1]:
#default_exp music_transformer.dataloader

In [2]:
#export
"Fastai Language Model Databunch modified to work with music"

'Fastai Language Model Databunch modified to work with music'

In [3]:
#export
from musicautobot.imports import *
from musicautobot.music_transformer.transform import *
from musicautobot.vocab import MusicVocab

import random
from fastai.basics import *
from fastai.text.all import *

## Text Model

In [10]:
# path = untar_data(URLs.WIKITEXT_TINY)
# df_train = pd.read_csv(path/'train.csv', header=None)
# df_valid = pd.read_csv(path/'test.csv', header=None)
# all_texts = np.concatenate([df_train[0].values, df_valid[0].values])

# from fastai.text.all import *

# class TransformersTokenizer(Transform):
#     def __init__(self, tokenizer): self.tokenizer = tokenizer
#     def encodes(self, x): 
#         toks = self.tokenizer.tokenize(x)
#         return tensor(self.tokenizer.convert_tokens_to_ids(toks))
#     def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

# from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# pretrained_weights = 'gpt2'
# tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)

# splits = [range_of(df_train), list(range(len(df_train), len(all_texts)))]
# tls = TfmdLists(all_texts, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)

# bs,sl = 4,256
# dls = tls.dataloaders(bs=bs, seq_len=sl)

# dls.train

## Music Model

In [4]:
base_path = Path('../../data')
midi_path = base_path/'midi/examples'

In [5]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)
vocab = MusicVocab.create()

In [6]:
#export
class Midi2ItemTfm(Transform):
    "Skips midi preprocessing step. And encodes midi files to MusicItems"
    def __init__(self,vocab):
        self.vocab = vocab
    
    def encodes(self, o):
        return MusicItem.from_file(o, vocab=self.vocab)
    
class MusicItemTfm(Transform):
    "`PreProcessor` that transforms numpy files to indexes for training"
    def __init__(self,vocab):
        self.vocab = vocab
        
    def encodes(self, f):
        npitem = np.load(f, allow_pickle=True) if isinstance(f, Path) else f
        miitem = MusicItem.from_npenc(npitem, vocab=self.vocab)
        return miitem

In [7]:
#export
def mi2tensor(mi): return np.stack([mi.data, mi.position]).T

def rand_transpose(mi, steps=6, p=0.5):
    if random.random() >= p: return mi
    val = random.randint(-steps, steps)
    return mi.transpose(val)

def batch_position_tfm(b):
    "Batch transform for training with positional encoding"
    x,y = b
    x = {
        'x': x[...,0],
        'pos': x[...,1]
    }
    return x, y[...,0]

@delegates()
class MusicItemDataLoader(LMDataLoader):
    def __init__(self, dataset, trange=6, tp=0.5, encode_position=False, **kwargs):
        store_attr('trange,tp,encode_position')
        super().__init__(dataset, **kwargs)
        
    def make_chunks(self): 
        transpose_tfm = partial(rand_transpose, steps=self.trange, p=self.tp)
        tensor_tfm = mi2tensor if self.encode_position else lambda x: x.data
        pipeline = Pipeline([transpose_tfm, tensor_tfm])
        
        self.chunks = Chunks(list(map(pipeline, self.items)), self.lens)
        
    def create_item(self, seq):
        item = super().create_item(seq)
        return batch_position_tfm(item) if self.encode_position else item

## TODO: Make mi2tensor and rand_transpose into a pipeline transform.
## It no longer needs to be inside dataloader.

In [13]:
tfms = [Midi2ItemTfm(vocab)]
splits = RandomSplitter(seed=42)(range(len(midi_files)))
dsets = Datasets(midi_files, [tfms], splits=splits)

In [14]:
dls = DataLoaders.from_dsets(dsets, dl_type=MusicItemDataLoader, bs=1, seq_len=13)
xb,yb = dls.one_batch()
xb['x'][:,:10]

LMTensorText([[  0,   1,  71, 139,  66, 145,  62, 145,  59, 145]])

In [17]:
# # Sanity Check
# music_items = [MusicItem.from_file(f, vocab=vocab) for f in midi_files]
# tensor_items = [mi2tensor(mi) for mi in music_items]
# lens = [len(mi) for mi in music_items]
# cs = Chunks(tensor_items, lens)
# cs[1:10]

In [1]:
#hide
from nbdev.export import notebook2script
notebook2script(recursive=True)

Converted config.ipynb.
Converted dataloader.ipynb.
Converted learner.ipynb.
Converted model.ipynb.
Converted transform.ipynb.
Converted Train.ipynb.
Converted dataloader-reference.ipynb.
Converted dataloader-v1.ipynb.
Converted dataloader.ipynb.
Converted learner.ipynb.
Converted model.ipynb.
Converted transform.ipynb.
Converted numpy_encode.ipynb.
Converted attention_mask.ipynb.
Converted env_setup.ipynb.
Converted file_processing.ipynb.
Converted lamb.ipynb.
Converted midifile.ipynb.
Converted stacked_dataloader.ipynb.
Converted top_k_top_p.ipynb.
Converted vocab.ipynb.
